# there is the SHAPE dataset of 50 sequences, under 4 conditions... 

we want to see of our accessibility is good

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
import sys
sys.path.insert(0,'..')
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

import matplotlib.pyplot as plt
import simushape as ss 
from scipy.stats import spearmanr as spear
import pandas
import numpy as np


# Load data

In [ ]:
import rna_tools.rna_io as rio
import pandas

types = ["cellfree",
"incell",
"kasugamycin"]

def getdata(typ):
    return rio.get_all_data("../data/weeks194_orig/%s.react" % typ,"../data/weeks194_orig/%s.dbn" % typ)  # {key: rea, seq, stru}

data = [getdata(t) for t in types] # 2 has bad results :)


import seaborn as sns
def draw(data,ylabel="SHAPE reactivity",piclength=6, title="Experimental Reavtivity for mRNA#96",ylim=(0, 1)):
    fig, ax = plt.subplots()
    # the size of A4 paper
    fig.set_size_inches(piclength,4)
    
    sns.set(style="whitegrid")
    g=sns.lineplot(data=data.T, dashes=False)
    g.set_xlabel('Transcript')
    g.set_ylabel(ylabel)
    plt.title(title)
    g.set(ylim=ylim)
    #fig = g.get_figure()
    #fig.savefig("output.pdf")
    
def window(stuff):
    windowed = [ np.median(np.array( [z for z in stuff[i:i+51] if z != None] )) for i in range(0, len(stuff)-50)  ]
    return np.hstack([[np.nan]*25,windowed])  # add padding

# FIG 2   -- "weeks 50" paper  -- just shape profiles

In [ ]:
# GRAPH FROM PAPER 

papr = pandas.DataFrame( [ window(data[a]["rpsP_rimM_trmD_rplS_96"][0][:500]) for a in range(3)],index=types)

draw(papr)

#   

In [ ]:
%%time
# SHAKER RECONSTRUCTION   # GRADIENT BOOST OPTIMIZED

keys = data[0].keys()
keys.remove("rpsP_rimM_trmD_rplS_96")
keys_train = keys[:5]

modelz=[ss.make_xgbreg() for i in range(len(data))]
models = [ss.make_model(dat,keys_train,model=mod) for dat,mod in zip(data,modelz) ]

predictions = [ss.predict(mod, data[0]["rpsP_rimM_trmD_rplS_96"][1][:500]) for mod in models]
mystuff = pandas.DataFrame( [ window(p) for p in predictions],index=types)

draw(mystuff)

In [ ]:
%%time
# SHAKER RECONSTRUCTION   # FORREST REGRESSOR
keys = data[0].keys()
keys.remove("rpsP_rimM_trmD_rplS_96")
keys_train = keys[:5]

modelz=[ss.make_forestregressor() for i in range(len(data))]
models = [ss.make_model(dat,keys_train,model=mod) for dat,mod in zip(data,modelz) ]

predictions = [ss.predict(mod, data[0]["rpsP_rimM_trmD_rplS_96"][1][:500]) for mod in models]
mystuff = pandas.DataFrame( [ window(p) for p in predictions],index=types)

draw(mystuff)


# FIG 6   -- "weeks 50" paper --regulatory structures

In [ ]:
%%time

# SHAKER RECONSTRUCTION   # GRADIENT BOOST OPTIMIZED

keys = data[0].keys()
keys.remove("rpmB_rpmG_147")
keys_train = keys[:5]

modelz=[ss.make_xgbreg() for i in range(len(data))]
models = [ss.make_model(dat,keys_train,model=mod) for dat,mod in zip(data,modelz) ]

predictions = [ss.predict(mod, data[0]["rpmB_rpmG_147"][1]) for mod in models]

# predictions 
mystuff = pandas.DataFrame( [ window(p) for p in predictions],index=types)

draw(mystuff)



In [ ]:
# accessibility for prediction   and accessibility for real data

from rna_tools import rnaplfold
acc =  [ rnaplfold.rnaplfold(data[0]["rpmB_rpmG_147"][1], p) for p in predictions]

# accessibility windowed
mystuff = pandas.DataFrame(map(window,acc),index=types)
mystuff.T.plot.line()

# ACCESS OROGINAL 
papr = pandas.DataFrame( [ window(   rnaplfold.rnaplfold(data[a]["rpmB_rpmG_147"][1],data[a]["rpmB_rpmG_147"][0])  ) for a in range(3)],index=types)
papr.T.plot.line()

---

# FIG 2B    

In [ ]:
%%time
import data.weeks194_orig.remove_genes as d


# get data
keys = data[0].keys()
keys_predict = ["rpsP_rimM_trmD_rplS_96","dusB_fis_130"]
keys_train = [k for k in  keys if k not in keys_predict][:100]


# make model
models =[ss.make_xgbreg()  for dat in data]
models = [ ss.make_model(dat,keys_train, model=mod)for dat,mod in zip(data,models) ]




In [ ]:
%%time
# PREDICTION
from rna_tools.rnaplfold import rnaplfold
import notebook_helper as nb
asd=nb.getgenedict()


# predict
def predwrap(k,mod):
    seq = data[0][k][1]
    res = ss.predict(mod, seq[:600] )
    return res

def plfoldrap(k):
    seq = data[0][k][1]
    res = rnaplfold(seq[:600])
    return res


k = keys_predict[0]
predictions = [predwrap(k,mod)  for mod in models]
predictions.append(plfoldrap(k))



In [ ]:
blub(predictions)

In [ ]:
%%time
models_forest =[ss.make_forestregressor()  for dat in data]
models_forest = [ ss.make_model(dat,keys_train, model=mod)for dat,mod in zip(data,models) ]
k = keys_predict[0]
predictions_forest = [predwrap(k,mod)  for mod in models_forest]
predictions_forest.append(plfoldrap(k))
blub(predictions_forest)

In [ ]:

k="rpsP_rimM_trmD_rplS_96"
import copy

def blub(df):
    # data
    realdata2 = copy.deepcopy(df)
    realdata2 = [window(p) for p in realdata2]
    
    # genes 
    #realdata2.append(nb.get_genetrack(k, data[0],asd)[:600])  # 
    names, genes = nb.get_genetrack_multigene(k,data[0],asd, drawindex=(0,600))
    realdata2+=genes
    mystuff = pandas.DataFrame(realdata2,index=types+["plfold"]+names)
    draw(mystuff, piclength=12)

    
realdata = [data[i][k][0][:600] for i in range(3)]
realdata.append(plfoldrap(k))
    
blub(realdata)


# Fig 3d

In [ ]:
keys = data[0].keys()
keys_train = keys[:5]
keys_predict = keys[-5:]

models =[ss.make_xgbreg()  for dat in data]
models = [ ss.make_model(dat,keys_train, model=mod)for dat,mod in zip(data,models) ]


predictions = [[ss.predict(mod, data[0][k][1][:150]) for k in keys_predict] for mod in models]



In [ ]:
res = [np.array(p).mean(axis=0) for p in predictions]
mystuff = pandas.DataFrame([ window(p) for p in res],index=types)

draw(mystuff)


# DO THE SAME FOR GENESTARTSITE +-100

In [ ]:
%%time
import data.weeks194_orig.remove_genes as d



# get data
keys = data[0].keys()
print len(keys)
keys_train = keys[:100]
keys_predict = keys[-25:]




# make model
models = [ss.make_xgbreg() for e in range(len(data))]
models = [ ss.make_model(dat,keys_train, model=mod)for dat,mod in 
          zip(data,models) ]




In [ ]:
%%time
# PREDICTION
from rna_tools.rnaplfold import rnaplfold



# select gene start 
genstartdict = d.read_genes()


def cut_genestart(k,sequence):
    """there is a sequence(of anything), we center the startsite of the first gene and add padding
    when necessary.
    returns cut_sequence, padleft, padright"""
    """helper for the 2 functions below"""
    index = int(k.split("_")[-1])
    genestart = genstartdict[index][0][0] # [transcript_index gene_index start]
    geneend = genstartdict[index][0][1] # [transcript_index gene_index end]
    sequence = sequence[:geneend]  # sequence[:geneend]
    endpointindex = min(genestart+150, geneend) 
    pre = sequence[max(genestart-150,0):genestart] # everything before the genestart
    post =   sequence[genestart:min(endpointindex, len(sequence))] # after genestart
    
    if type(sequence) != str:
        pre = [e if e != None else np.nan for e in pre ]
        post = [e if e != None else np.nan for e in post ]
    
    answer= pre+post , abs(min(genestart-150,0)), max(genestart+150-len(sequence),0) 
    # check error
    if (len(answer[0])+answer[1]+answer[2])!= 300:
        print answer, k,sequence, geneend
    # track progress alittle biiit
    print ".",
    return answer


    
# predict
def predwrap(k,mod):
    """predicts for a transcript"""
    seq, padstart, padend = cut_genestart(k,data[0][k][1])
    
    res = ss.predict(mod, seq )
    a=np.full((padstart,), np.nan, dtype=float)
    b = np.full((padend,), np.nan, dtype=float)
    ret= np.hstack( [a,res, b] )
    return ret

def plfoldrap(k):
    """predicts for a transcript"""
    seq, padstart, padend = cut_genestart(k,data[0][k][1])
    res = rnaplfold ( seq )
    a=np.full((padstart,), np.nan, dtype=float)
    b = np.full((padend,), np.nan, dtype=float)
    ret= np.hstack( [a,res, b] )
    return ret


predictions = [[predwrap(k,mod) for k in keys_predict] for mod in models]
predictions.append([plfoldrap(k) for k in keys_predict])
# we want the mean at nucleodide possition [-150.. 150] over all sequences...
res = [np.nanmean(np.array(p),axis=0) for p in predictions]

# windowing to smoothe the plot
mystuff = pandas.DataFrame([ window(p) for p in res],index=types+["plfold"])

draw(mystuff)

In [ ]:
#Original DAta

# select gene start 
genstartdict = d.read_genes()


def plfoldrap(k):
    seq, padstart, padend = cut_genestart(k,data[0][k][1]) # this is the verison above
    res = rnaplfold ( seq )
    a=np.full((padstart,), np.nan, dtype=float)
    b = np.full((padend,), np.nan, dtype=float)
    ret= np.hstack( [a,res, b] )
    return ret

# predict
def predwrap(k, exp):
    seq, padstart, padend = cut_genestart(k,data[exp][k][0])
    res =  np.array(seq) 
    a=np.full((padstart,), np.nan, dtype=float)
    b = np.full((padend,), np.nan, dtype=float)
    ret= np.hstack( [a,res, b] )
    return ret

predictions = [[predwrap(k,mod) for k in keys_predict] for mod in range(3)]
predictions.append([plfoldrap(k) for k in keys_predict])
# we want the mean at nucleodide possition [-150.. 150] over all sequences...
#res = [np.array(p).mean(axis=0) for p in predictions]

res = [np.nanmean(np.array(p),axis=0) for p in predictions]


# windowing to smoothe the plot
mystuff = pandas.DataFrame([ window(p) for p in res],index=types+["plfold"])
draw(mystuff)